## Simulation Initialization

In [ ]:
# We import these lines for debugging purposes, only works on Jupyter Notebook
%load_ext autoreload
%autoreload 2
%matplotlib widget

import datetime
from rocketpy import Environment, Flight, Function, MonteCarlo, Rocket, SolidMotor
from rocketpy.stochastic import (
    StochasticEnvironment,
    StochasticFlight,
    StochasticNoseCone,
    StochasticParachute,
    StochasticRailButtons,
    StochasticRocket,
    StochasticSolidMotor,
    StochasticTail,
    StochasticTrapezoidalFins,
)

In [ ]:
# Environment Configuration
env = Environment(latitude=31.04434, longitude=-103.53517, elevation=1400)

import datetime
tomorrow = datetime.date.today() + datetime.timedelta(days=1)

env.set_date(
    (tomorrow.year, tomorrow.month, tomorrow.day, 12)
)

env.set_atmospheric_model(type="Forecast", file="GFS")
env.info()

In [ ]:
# Motor Definition

K2FullMotor = SolidMotor(
    thrust_source = r".\Raw Data\Valkyrie\K-2FullScaleTest.eng",
    dry_mass = 5.44311,
    dry_inertia = (0.475, 0.475, 0.007),
    nozzle_radius = 0.06858/2,
    grain_number = 6,
    grain_density = 1501.44939, #calculated from Matt's sim
    grain_outer_radius = 0.082296/2,
    grain_initial_inner_radius = 0.0381/2,
    grain_initial_height = 0.147955,
    grain_separation = 0,
    grains_center_of_mass_position=0.508,
    center_of_dry_mass_position=0.4826,
    nozzle_position=0,
    burn_time = 4.06,
    throat_radius = 0.02794/2,
    coordinate_system_orientation="nozzle_to_combustion_chamber"
)


In [ ]:
# Define Rocket
Valkyrie = Rocket(
    radius = 0.078359,
    mass = 20.61,
    inertia = (0.09, 12.34, 12.34),
    power_off_drag="Raw Data/Valkyrie/SolidWorks_Power Off Drag Curve.csv",
    power_on_drag="Raw Data/Valkyrie/SolidWorks_Power Off Drag Curve.csv",
    center_of_mass_without_motor=1.92,
    coordinate_system_orientation="nose_to_tail",
)
Valkyrie.add_motor(K2FullMotor, position =3.552)

rail_buttons = Valkyrie.set_rail_buttons(
    upper_button_position=3,
    lower_button_position = 2,
    angular_position=0,
)

fin_set = Valkyrie.add_trapezoidal_fins(
    n=4,
    root_chord=0.3048,
    tip_chord = 0.0767,
    span = 0.117,
    position = 3.247,
    cant_angle = 0,
    sweep_angle = 59.44,
)

nose_cone = Valkyrie.add_nose(
    length = 0.784, kind="von karman", position = 0
)

Drogue = Valkyrie.add_parachute(
    "Drogue", cd_s=1.00584, trigger="apogee", sampling_rate=100, lag=0
)

Main = Valkyrie.add_parachute(
    "Main", cd_s=2.34696, trigger=304.8, sampling_rate=100, lag=0
)
Valkyrie.draw()

In [ ]:
# Nominal Flight Simulation
test_flight = Flight(
    rocket = Valkyrie, environment=env, rail_length=5.1, inclination=84.2, heading = 135)
test_flight.info()
test_flight.plots.attitude_data()
test_flight.plots.trajectory_3d()

# Begin Stochastic Modeling of Rocket

In [ ]:
# Stochastic Environment

stochastic_env = StochasticEnvironment(
    environment = env,
)

stochastic_env.visualize_attributes()

In [ ]:
#  Stochastic Motor
stochastic_motor = StochasticSolidMotor(
    solid_motor=K2FullMotor,
    burn_start_time = (0, 0.1, "binomial"),
    grains_center_of_mass_position=0.001,
    dry_mass = 0.96,
    grain_separation=1/1000,
    grain_initial_height = 1 /1000,
    throat_radius = 0.5/1000,
    nozzle_radius = 0.5/1000,
    nozzle_position=0.001,
)
stochastic_motor.visualize_attributes()

In [ ]:
# Stochastic Rocket

stochastic_rocket = StochasticRocket(
    rocket = Valkyrie,
    radius = 0.0127/1000,
    mass = 0.24,
    power_off_drag_factor= (1,0.1),
    power_on_drag_factor=(1,0.1),
)

stochastic_rocket.add_motor(stochastic_motor, position=0.001),
stochastic_rocket.visualize_attributes()

In [ ]:
# Stochastic Flight
stochastic_flight = StochasticFlight(
    flight = test_flight,
    inclination = 2.5,
    heading = 10,
)
stochastic_flight.visualize_attributes()

In [ ]:
# Monte Carlo Simulation
test_dispersion = MonteCarlo(
    filename = "MonteCarlo/ValkyrieMC",
    environment=stochastic_env,
    rocket=stochastic_rocket,
    flight=stochastic_flight,
)

test_dispersion.simulate(
    number_of_simulations=500,
    append=False,
    include_function_data=False,
    parallel=True,
    n_workers = 10,
)

In [ ]:
test_dispersion.num_of_loaded_sims
test_dispersion.prints.all()

In [ ]:
test_dispersion.plots.all()